**Разбивка на подзадачи**:
- подготовка данных
- методика расчета средних охватов по станции (15мин, день, неделя, месяц)
- методика расчета суммарного рейтинга GRP
- методика расчета Reach 1+ для однодневной рекламной кампании (РК)  
- методика Reach 1+ для расчета РК по нескольким дням внутри одной недели (пересечение однодневных охватов)
- методика расчета частотного охвата

In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib
import seaborn as sns
import scipy as sc
import functions
import warnings
warnings.filterwarnings ( action='ignore')

In [2]:
# 1. Подготовка данных

Исходные данные представлены в виде 2 таблиц: в одной закодированы дневники слушания по каждому дню недели в разрезе 15 минуток, во второй закодированы соц-дем профиль, в том числе ответы на вопросы о частоте слушания той или иной радиостанции.  
Исследование проводилось в г. Москва в 1 кв 2021г методом телефонного опроса Day-After-Recall, и включает результаты опроса 15 138 респондентов.   
Таблицу с дневниками слушания загрузим полностью, таблица с соц-демом содержит 500 с лишним колонок, поэтому для расчетов возьмем выборку из этой таблицы, которая содержит данные с ответами на вопросы о частоте слушания Авторадио.



In [3]:
# Дневники слушания (сведенные в одну таблицу)
diary = pd.read_csv ('./src/data/diary.csv', sep=';')
diary

,Member_nr,day,Chl_id,weights,1,2,3,4,5,6,...,87,88,89,90,91,92,93,94,95,96
0,1000000001,3,152,0.6037,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000000002,5,112,1.1176,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1000000004,7,115,1.0779,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1000000004,7,199,1.0779,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1000000005,5,101,1.0603,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26403,1000015138,4,127,0.7903,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26404,1000015138,4,164,0.7903,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26405,1000015138,4,170,0.7903,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26406,1000015138,4,179,0.7903,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Описание колонок:  
**Member_nr** - уникальный ID респондента  
**day** - закодированный день недели слушания  
**Chl_id** - код радиостанции   
**weights** - веса респондентов (в методиках часто используется другое обозначение *projection factor*), это количество людей в ген. совокупности соотвествующее одной весовой единице в опросе. Проще говоря, это поправочный коэфициент, который позволяет выровнять распредление основных соц-дем характеристик (пол, возраст, размер семьи) между выборкой и ген. совокупностью.      
**1...96** - номера слотов (15 минутных интервалов в сутках), в которых указан рейтинг слушания:  
- ноль, если респондент не слушал ни одной радиостанции в данном 15-мин интервале
- единица деленная на количество станций, которые слушал респондент в данном 15-мин интервале, в противном случае    

In [4]:
# Выборка из соц-дем таблицы
rt = pd.read_csv ('./src/data/data.csv', sep=';')
rt

,ID,w,day_list,week_list,month_list
0,1000000001,0.6037,0,0,0
1,1000000002,1.1176,0,0,0
2,1000000003,0.8871,0,0,0
3,1000000004,1.0779,0,0,0
4,1000000005,1.0603,1,7,1
...,...,...,...,...,...
15133,1000015134,1.7235,1,7,1
15134,1000015135,0.8765,0,6,1
15135,1000015136,0.8866,0,0,0
15136,1000015137,0.8601,0,0,1


Описание колонок:  
**ID** - уникальный ID респондента  
**w** - веса респондентов  
**day_list** - индикатор суточного слушания "Авторадио"   
**week_list** - индикатор неедельного слушания "Авторадио" (количество дней недели)   
**month_list** - индикатор месячного слушания "Авторадио" 

В качестве бенчмарка используем 3 медиаплана, рассчитанные в SuperNova MediaScope по следующим вводным:    
**База данных:** Radio Index - Moscow. January - March 2021  
**Размер генеральной совокупности (тыс.):** 10 973,3  
**Размер целевой группы (тыс.):** 10 973,3  
**Выборка:** 15 138    
**Целевые медиа:** Авторадио  
**МП1** - 2 выхода в час, 4 выхода в день, с 10 до 11, с 12 до 13, с 14 до 15, с 18 до 19 часов, период: понедельник (одна неделя), итого - 8 выходов  
**МП2** - 2 выхода в час, 4 выхода в день, с 10 до 11, с 12 до 13, с 14 до 15, с 18 до 19 часов, период: понедельник-пятница  (одна неделя), итого - 40 выходов  
**МП3** - 4 выхода в час, 3 выхода в день, с 15 до 18 часов, период: понедельник-воскресенье  (четыре недели), итого - 336 выходов  

In [5]:
mp1 = pd.read_csv ('./src/data/mp1.csv', sep=';').dropna()
mp2 = pd.read_csv ('./src/data/mp2.csv', sep=';').dropna()
mp3 = pd.read_csv ('./src/data/mp3.csv', sep=';').dropna()
mp1

,Показатель,Авторадио
0,GI,555.88
1,Frequency,1.82
2,Index T/U Reach,100.00
3,GRP,5.07
4,TRP,5.07
5,RP Index,100.00
6,Spots,8.00
7,Reach 1+,306.13
8,Reach 2+,120.83
9,Reach 3+,58.34


На последнем этапе подготовки зафиксируем переменные среды, так как вызовы функций настроены на входные данные, которые должны содержать разные таргетинги, разные станции и разное расписание выходов, а мы работаем с зафиксированными вводными, чтобы каждый раз не передавать эти данные, запишем их в константы

In [6]:
CH = [101] # код Авторадио
AUD = rt['ID'].to_list() # ID целевой аудитории в виде списка, в нашем случае это все ID
MP1_D = [1] # номер дня недели выхода ролика
MP2_D = [1,2,3,4,5]
MP3_D = [1,2,3,4,5,6,7]
MP1_SCH = [10,12,14,18] # часы выходов
MP2_SCH = MP1_SCH
MP3_SCH = [15,16,17]
MP1_N = 2 # количество выходов в час
MP2_N = MP1_N
MP3_N = 4

# 2.  Подсчет аудиторий станции (15 мин, день, неделя, месяц)

Расчет рейтинга 15 минутки идет по формуле:
$\sum_{i = 1}^{n}\dfrac{w_i*t_i}{Day\_Aud}$, где 
- $n$ - количество респондентов, имеюших записи в дневнике слушания в выбранный день 
- $w_i$ веса всех респондентов, присутствующих в дневнике слушания в выбранный день
- $t_i$ рейтинг слушания выбранной радиостанции в интервале 15 минут (значение от нуля до единцы, в зависимости от количества слушаемых станций) 
- $Day\_Aud$ сумма весов респондентов, опрошенных в выбранный день  
Последнее значение в расчетах можно принять за константу равную $\frac{15318}{7}$, так как волна опроса соотвествующим образом была равномерно распределена по дням недели

In [7]:
# Итоговая функция для подсчета рейтинга 15-минутки
# считаем на примере: понедельник, Авторадио, интервал: 14:00 - 14:15 
# нумерация 15-минуток в дневниках слушания начинается с 5:00 утра, так что времени 14:00 соответствует 37 слот
round (functions.reach_slot(AUD, MP1_D[0], CH[0] ,'37')*100, 2)

0.91

Теперь подсчитаем показатель AQH (средний рейтинг 15-минутки) для Авторадио, по методике для этого надо рассчитать рейтинги всех 15 минуток по всем 7 дням недели и усреднить их.  
При этом в отчете MediaScope за 1 кв 2021г, значения AQH даны для диапазона 06:00-24:00, поэтому нам также нужны значения только для этого интервала времени 

In [8]:
# AQH Авторадио
res=[]
for day in range (1,8):
    for slot in range (5, 77):
        res.append (functions.reach_slot ( AUD, day, CH[0], str(slot)))
print ('AQH Авторадио -', round (np.array(res).mean() ,3) *100) 

AQH Авторадио - 0.5


Сравниваем с бенчмарком - все совпало
<img src='./src/images/AQH.png'>

Теперь перейдем к расчету средних рейтингов: **Reach Daily**, **Reach Weekly**, **Reach Monthly**  
Они определяются очень просто - как доля положительно ответивших на соответствующий вопрос (с учетом весов) по отношению к общему количеству опрошенных.   
Получим соотвествующие значения для Авторадио

In [9]:
print ('Средний дневной охват "Авторадио", Daily Reach-', round(np.dot(rt['day_list'],rt['w'])/15138 *100, 2 )) 
print ('Средний недельный охват "Авторадио", Weekly Reach -', round(np.dot(np.where (rt['week_list']==0,0,1),rt['w'])/15138*100,1))
print ('Средний месячный охват "Авторадио", Monthly Reach -', round(np.dot(rt['month_list'],rt['w'])/15138 *100, 2 )) 

Средний дневной охват "Авторадио", Daily Reach- 9.1
Средний недельный охват "Авторадио", Weekly Reach - 25.2
Средний месячный охват "Авторадио", Monthly Reach - 34.55


Расчеты верны, можем смело опираться на эти формулы в дальнейшем

# 3. Расчет GRP

Переходим к расчетам медиапоказателей РК, начнем с самого простого: подсчету GRP/TRP.  
Формула расчета GRP:  
$GRP = \sum_{d}\sum_{i = 1}^{k} Reach\_slot_i*\dfrac{n_i}{4}$, где
- $d$ - дни выходов ролика в течении РК
- $k$ - количество слотов в которых есть выходы ролика
- $Reach\_slot$ - рейтинг 15-минутки
- $n$ - количество выходов ролика в час  
По сути это просто сумма рейтингов 15-минуток, с поправкой на количество выходов в час. При медиапланировании РК на радио минимальным периодом планирования является час и ролики равномерно распределяются по сетке выходов, соответственно, если количество выходов в час равно 1, то рассматривая эту ситуацию с точки зрения теории вероятности, мы понимаем что, с вероятностью 0,25 выход ролика попадет в любой из 4 слотов, итоговую вероятность по каждому респонденту надо оценивать как 
сумму рейтингов слушания 15 минуток с коэфициентом 0,25. Если будет 2 выхода в час, то 1 ролик выйдет в первые полчаса (первые 2 слота часа), а ворой ролик во вторые полчаса (3,4 слот часа). Соответственно, первый ролик с вероятностью 0,5 попадет в 1 слот, либо с вероятностью 0,5 во второй. Аналогично со вторым роликом и вероятностями попасть в 3 и 4 слоты. В итоге получается формула для расчета поправочного коэфициента $\dfrac{n}{4}$  

*Примечание*: поскольку мы изначально зафиксировали в качестве целевой аудитории всех респондентов, то во-первых, в нашем случае не надо считать TRP, поскольку GRP=TRP, во-вторых, все приведенные формулы даны, с учетом этого, в упрощенном варианте. На практике при расчете рейтинга 15-минутки в формуле GRP нужно в числителе брать только веса респондентов из целевой аудитории (знаменатель остается без изменений), а для расчета TRP, нужно еще и в знаменателе в оставить в сумме только веса респондентов, принадлежащих целевой аудитории. Формулы для расчета GRP, TRP в модуле functions как раз реализуют полную функциональность.  

In [10]:
print (round (functions.GRP (diary, CH[0], MP1_D[0], MP1_N , MP1_SCH ),2 ))

5.07


In [ ]:
functions.grp (AUD, CH, MP2_D, MP2_SCH, MP2_N)

In [ ]:
functions.WR_cum (AUD, CH, MP2_D, MP2_SCH, MP2_N)

functions.WR_cum (AUD, CH, MP1_D, MP1_SCH, MP1_N)

In [ ]:
mp2

functions.WR_cum (AUD, CH, MP1_D, MP1_SCH, MP1_N)

In [13]:
mp2

,Показатель,Авторадио
0,GI,2847.30
1,Frequency,2.10
2,Index T/U Reach,100.00
3,GRP,25.90
4,TRP,25.90
5,RP Index,100.00
6,Spots,40.00
7,Reach 1+,1346.52
8,Reach 2+,624.68
9,Reach 3+,340.99


# 4. Расчет Reach 1+ 